In [1]:
from tensorflow.keras.datasets.mnist import load_data
import tensorflow as tf
import numpy as np


(X_train, y_train), (X_test, y_test) = load_data()

In [2]:
X_train = X_train / 255.
X_test = X_test / 255.

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

X_train = X_train.reshape([*X_train.shape, 1])
X_test = X_test.reshape([*X_test.shape, 1])

In [3]:
learning_rate = 0.001
training_iters = 1000
batch_size = 128
display_step = 10

In [4]:
n_input = 784 # MNIST data input (shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [5]:
# Dropout
# Aplicamos o dropout para reduzir o overfitting. O dropout vai eliminar algumas unidades (nas camadas ocultas, de entrada e de saída) na rede neural.
# A decisão sobre qual neurônio será eliminado é randômica e aplicamos uma probabilidade para isso. Esse parâmetro pode ser ajustado para otimizar o desempenho da rede.
dropout = 0.75 # Dropout, probabilidade para manter unidades

In [6]:
print(X_train.shape)
print(y_train.shape)

(60000, 28, 28, 1)
(60000,)


In [7]:
# Variáveis para os pesos e bias

# Pesos
# Cada neurônio da camada oculta é conectado a um pequeno grupo de tensores de entrada (input) de dimensão 5x5. Com isso, a camada oculta terá um tamanho de 24x24.
wc1 = tf.Variable(tf.random.normal([5, 5, 1, 32])) # 5x5 conv, 1 input, 32 outputs
wc2 = tf.Variable(tf.random.normal([5, 5, 32, 64])) # 5x5 conv, 32 inputs, 64 outputs
wd1 = tf.Variable(tf.random.normal([4*4*64, 1024])) # fully connected, 7*7*64 inputs, 1024 outputs
wout = tf.Variable(tf.random.normal([1024, n_classes])) # 1024 inputs, 10 outputs (class prediction)

# Bias
bc1 = tf.Variable(tf.random.normal([32]))
bc2 = tf.Variable(tf.random.normal([64]))
bd1 = tf.Variable(tf.random.normal([1024]))
bout = tf.Variable(tf.random.normal([n_classes]))

In [8]:
# @tf.function
def conv2d(image, w, b):
    conv = tf.nn.conv2d(image, filters=w, strides=[1,1,1,1], padding='VALID')
    conv = tf.nn.bias_add(conv, b)
    return tf.nn.relu(conv)

In [9]:
# @tf.function
def max_pool(image, k):
    return tf.nn.max_pool(image, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='VALID')

In [10]:
# @tf.function
def full_connected(input, w, b):
    return tf.add(tf.matmul(input, w), b)

In [11]:
# @tf.function
def cross_entropy(y_true, y_pred):
    y_true = tf.one_hot(y_true, n_classes)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [12]:
# Cria o otimizador usando Adam
optimizer = tf.optimizers.Adam(learning_rate)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.repeat().shuffle(X_train.shape[0]).batch(batch_size)

In [14]:
# @tf.function
def cnn_net(X):
    conv1 = conv2d(X, wc1, bc1)
    conv1 = max_pool(conv1, k=2)
    conv1 = tf.nn.dropout(conv1, 1 - dropout)

    conv2 = conv2d(conv1, wc2, bc2)
    conv2 = max_pool(conv2, k=2)
    conv2 = tf.nn.dropout(conv2, 1 - dropout)

    dense1 = tf.reshape(conv2, [-1, wd1.shape[0]])
    dense1 = full_connected(dense1, wd1, bd1)
    dense1 = tf.nn.relu(dense1)

    pred = full_connected(dense1, wout, bout)
    
    return pred


In [15]:
def grad(X, y):
    with tf.GradientTape() as tape:
        pred = cnn_net(X)
        loss = cross_entropy(y, pred)
    return tape.gradient(loss, [wc1, bc1, wc2, bc2, wd1, bd1, wout, bout])

In [16]:
def accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, dtype = tf.int32)
    preds = tf.cast(tf.argmax(y_pred, axis = 1), dtype = tf.int32)
    preds = tf.equal(y_true, preds)
    return tf.reduce_mean(tf.cast(preds, dtype = tf.float32))

In [17]:
for i, (batch_X_train, batch_y_train) in enumerate(dataset.take(training_iters)):
   
    gradient = grad(batch_X_train, batch_y_train)
    optimizer.apply_gradients(zip(gradient, [wc1, bc1, wc2, bc2, wd1, bd1, wout, bout]))
    
    y_pred = cnn_net(batch_X_train)

    # Calcula o erro
    loss = cross_entropy(batch_y_train, y_pred)

    # Calcula a acurácia
    acc = accuracy(batch_y_train, y_pred)

    # Print
    print("Número do Batch: %i, Erro do Modelo: %f, Acurácia em Treino: %f" % (i, loss, acc))


Número do Batch: 0, Erro do Modelo: 58723.597656, Acurácia em Treino: 0.078125
Número do Batch: 1, Erro do Modelo: 60254.902344, Acurácia em Treino: 0.078125
Número do Batch: 2, Erro do Modelo: 48314.953125, Acurácia em Treino: 0.125000
Número do Batch: 3, Erro do Modelo: 45148.031250, Acurácia em Treino: 0.117188
Número do Batch: 4, Erro do Modelo: 45903.398438, Acurácia em Treino: 0.148438
Número do Batch: 5, Erro do Modelo: 45112.335938, Acurácia em Treino: 0.171875
Número do Batch: 6, Erro do Modelo: 50048.429688, Acurácia em Treino: 0.078125
Número do Batch: 7, Erro do Modelo: 43155.511719, Acurácia em Treino: 0.156250
Número do Batch: 8, Erro do Modelo: 41680.675781, Acurácia em Treino: 0.117188
Número do Batch: 9, Erro do Modelo: 37689.109375, Acurácia em Treino: 0.148438
Número do Batch: 10, Erro do Modelo: 40694.781250, Acurácia em Treino: 0.132812
Número do Batch: 11, Erro do Modelo: 36219.101562, Acurácia em Treino: 0.171875
Número do Batch: 12, Erro do Modelo: 35388.000000,

In [18]:
# Testando o Modelo

# Preparando os dados de teste
dataset_teste = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dataset_teste = dataset_teste.repeat().shuffle(X_test.shape[0]).batch(batch_size)

print ("\nIniciando a Avaliação com Dados de Teste. Por favor aguarde!")

# Loop pelos dados de teste, previsões e cálculo da acurácia
for batch_numb, (batch_xs_teste, batch_ys_teste) in enumerate(dataset_teste.take(training_iters), 1):
    y_pred = cnn_net(batch_xs_teste)
    acc = accuracy(batch_ys_teste, y_pred)
    acuracia = tf.reduce_mean(tf.cast(acc, tf.float64))

print("\nAcurácia em Teste: %f" % acuracia)


Iniciando a Avaliação com Dados de Teste. Por favor aguarde!

Acurácia em Teste: 0.937500


In [20]:
print("\nFazendo Previsão de Uma Imagem:")

# Obtendo os dados de algumas imagens
dataset_teste = tf.data.Dataset.from_tensor_slices((X_test, y_test))
dataset_teste = dataset_teste.repeat().shuffle(X_test.shape[0]).batch(10)

# Fazendo previsões
for batch_numb, (batch_xs, batch_ys) in enumerate(dataset_teste.take(10), 1):
    # print("\nImagem:", batch_xs)
    print("\nClasse Real:", batch_ys)
    y_pred = tf.math.argmax(cnn_net(batch_xs), axis = 1)
    #y_pred = logistic_regression(batch_xs)
    print("Classe Prevista:", y_pred)


Fazendo Previsão de Uma Imagem:

Classe Real: tf.Tensor([4 7 6 0 8 8 0 4 3 8], shape=(10,), dtype=uint8)
Classe Prevista: tf.Tensor([4 7 6 0 8 8 0 6 3 8], shape=(10,), dtype=int64)

Classe Real: tf.Tensor([2 6 5 0 4 9 5 7 9 0], shape=(10,), dtype=uint8)
Classe Prevista: tf.Tensor([2 6 5 0 4 9 5 7 9 0], shape=(10,), dtype=int64)

Classe Real: tf.Tensor([7 9 5 1 6 0 8 8 8 2], shape=(10,), dtype=uint8)
Classe Prevista: tf.Tensor([7 9 5 1 6 0 9 8 8 2], shape=(10,), dtype=int64)

Classe Real: tf.Tensor([2 9 1 4 9 9 9 7 9 1], shape=(10,), dtype=uint8)
Classe Prevista: tf.Tensor([2 9 1 4 9 9 9 7 9 1], shape=(10,), dtype=int64)

Classe Real: tf.Tensor([8 9 0 8 5 0 5 8 3 4], shape=(10,), dtype=uint8)
Classe Prevista: tf.Tensor([8 9 0 9 5 0 5 9 3 4], shape=(10,), dtype=int64)

Classe Real: tf.Tensor([3 2 2 3 1 8 3 6 0 9], shape=(10,), dtype=uint8)
Classe Prevista: tf.Tensor([3 2 2 3 1 8 3 6 0 9], shape=(10,), dtype=int64)

Classe Real: tf.Tensor([3 4 8 4 3 0 4 7 4 2], shape=(10,), dtype=uint8)
